In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-xz9w1xga
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-xz9w1xga
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=177198e95603de3566f3f7e4c34f632abc2928c92893b21a17a3c1a34008e652
  Stored in directory: /tmp/pip-ephem-wheel-cache-m8xiafe6/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [6]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void sum_avg_kernel(int *data, int *sum, int *avg) {
  
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int s = 0;
  for (int j = i; j < i + 10; j++) {
    s += data[j];
  }
  sum[blockIdx.x] = s;
  avg[blockIdx.x] = s / 10;
}

int main() {
  
  int data[100];
  int sum[10];
  int avg[10];
  
  for (int i = 0; i < 100; i++) {
    data[i] = i + 1;
  }
  
  int *dev_data;
  cudaMalloc((void**)&dev_data, 100 * sizeof(int));
  cudaMemcpy(dev_data, data, 100 * sizeof(int), cudaMemcpyHostToDevice);
  
  int *dev_sum;
  cudaMalloc((void**)&dev_sum, 10 * sizeof(int));
  
  int *dev_avg;
  cudaMalloc((void**)&dev_avg, 10 * sizeof(int));
  
  sum_avg_kernel<<<10, 10>>>(dev_data, dev_sum, dev_avg);
  
  cudaMemcpy(sum, dev_sum, 10 * sizeof(int), cudaMemcpyDeviceToHost);
  cudaMemcpy(avg, dev_avg, 10 * sizeof(int), cudaMemcpyDeviceToHost);
  
  for (int i = 0; i < 10; i++) {
    printf("Blok: %d Suma: %d, Srednia: %d\n", i, sum[i], avg[i]);
  }
  
  cudaFree(dev_data);
  cudaFree(dev_sum);
  cudaFree(dev_avg);
  
  return 0;
}

Blok: 0 Suma: 55, Srednia: 5
Blok: 1 Suma: 155, Srednia: 15
Blok: 2 Suma: 255, Srednia: 25
Blok: 3 Suma: 355, Srednia: 35
Blok: 4 Suma: 455, Srednia: 45
Blok: 5 Suma: 555, Srednia: 55
Blok: 6 Suma: 655, Srednia: 65
Blok: 7 Suma: 755, Srednia: 75
Blok: 8 Suma: 855, Srednia: 85
Blok: 9 Suma: 955, Srednia: 95

